# Scraping 20Min Ro

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import twython

datestring = time.strftime("%Y-%h-%d")
datestring

'2016-Jul-10'

In [2]:
api_key = ""
api_secret = ""

access_token = ""
token_secret = ""

In [3]:
def to_int(x):
    if x == None:
        return 0
    else:
        return int(x)

In [4]:
twitter = twython.Twython(api_key, api_secret, access_token, token_secret)

In [5]:
# Grab the 20Min Homepage
response_20_minutes = requests.get("http://www.20min.ch/ro/")

In [6]:
# Feed it into BeautifulSoup
_20Minutes = BeautifulSoup(response_20_minutes.text, 'html.parser')

In [7]:
stories_20Minutes = _20Minutes.find_all("div", {'id': 'content'})

In [8]:
#finding everything outside of the loop
teaser_stories_20Minutes = stories_20Minutes[0].find_all('div', {'class': "teaser "})

#starting the dictionary:
all_stories_20Minutes = []

#The Loop
for story in teaser_stories_20Minutes:
    
    #Getting the comments
    comments = story.find('div', {'class': "teaser_media front clearfix"})
    if comments:
        try:
            comment = comments.find('span', {'class': 'icon_talkback'}).next_sibling
            #Just using the number of comments
            comment = comment[:4]
            #A regular expression to make sure to get articles with more of 999 comments
            if re.search(r"\D$", comment):
                comment = int(comment[:3])
            else:
                comment = int(comment)
        except:
            comment = 1
    
    #Getting the the title
    title_div = story.find('div', {'class': "teaser_title front"})
    title_string = title_div.find('h2').string
    if re.search(r"\x96", title_string):
        title_string.replace("\x96", "")
        
    
    #Getting the HREF
    #href = story.find('h3')
    #href = href.find('a', href=True)
    #href = href['href']
    
    #Dealing with non-full links
    #if re.search(r"^http:", href):
    #    href = href
    #else:
    try:
        href = story.find('h3').get('data-vr-contentbox')
        href = href.replace('Link ', 'http://www.20min.ch')
    except:
        0
    #Creating the dictionary:
    dictionary = {'Site': '@20minutesonline', 'Titel': title_string, 'Kommentare': to_int(comment), 'Link': href, 'Date': datestring}
    
    #List of dictionaries:
    all_stories_20Minutes.append(dictionary)


In [9]:
#all_stories_20Minutes

In [10]:
#sorting the dictionary according to the value of the keys
import operator
all_stories_20Minutes.sort(key=operator.itemgetter('Kommentare'), reverse=True)

In [11]:
#all_stories_20Minutes

In [12]:
sentence = "En Romandie, l'article le plus commenté est par {}, avec {} commentaires. {}".format(all_stories_20Minutes[0]['Site'], all_stories_20Minutes[0]['Kommentare'],  all_stories_20Minutes[0]['Link'])

In [13]:
sentence

"En Romandie, l'article le plus commenté est par @20minutesonline, avec 345 commentaires. http://www.20min.ch/ro/news/vaud/story/Des-militants-transformes-en-barquettes-de-viande-23495719"

In [14]:
twitter.update_status(status=sentence)

{'contributors': None,
 'coordinates': None,
 'created_at': 'Sun Jul 10 12:31:59 +0000 2016',
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [{'display_url': '20min.ch/ro/news/vaud/s…',
    'expanded_url': 'http://www.20min.ch/ro/news/vaud/story/Des-militants-transformes-en-barquettes-de-viande-23495719',
    'indices': [89, 112],
    'url': 'https://t.co/I8nyT5ItOk'}],
  'user_mentions': [{'id': 123840635,
    'id_str': '123840635',
    'indices': [48, 64],
    'name': '20 minutes',
    'screen_name': '20minutesOnline'}]},
 'favorite_count': 0,
 'favorited': False,
 'geo': None,
 'id': 752118165564424192,
 'id_str': '752118165564424192',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'fr',
 'place': None,
 'possibly_sensitive': False,
 'retweet_count': 0,
 'retweeted': False,
 'source': '<a href="http://www.barnabyskinner.